In [1]:
import numpy as np
import pandas as pd
import cv2
from sldata import SessionData

### Load Bonsai Data (Now with SLEAP) 

In [2]:
mouse_id = "7013"
session_id = "m13"
experiment = "clickbait-motivate"
base_path = "D:/data/"

print("=== Testing SessionData Class ===")
print(f"Loading data for {mouse_id}_{session_id}...")

data = SessionData(
    mouse_id=mouse_id,
    session_id=session_id,
    experiment=experiment,
    base_path=base_path,
    min_spikes=100,
    verbose=True)

=== Testing SessionData Class ===
Loading data for 7013_m13...
Successfully loaded spike_times from D:\data\clickbait-motivate\kilosorted\7013\m13\spike_times.npy
Successfully loaded spike_templates from D:\data\clickbait-motivate\kilosorted\7013\m13\spike_templates.npy
Successfully loaded templates from D:\data\clickbait-motivate\kilosorted\7013\m13\templates.npy
Successfully loaded sniff from D:\data\clickbait-motivate\preprocessed\7013\m13\sniff.npy
Loaded data: ['spike_times', 'spike_templates', 'templates', 'sniff']
Successfully loaded events from D:\data\clickbait-motivate\bonsai\7013\m13\events.csv
Video properties: 888x1968, 30.0 FPS, 81577 frames
Loaded cluster labels: 11 good, 32 mua, 7 noise
Excluded 7 noise clusters
Found 21709 peaks in sniff signal
Peak times range: 0.8 - 91310.7 ms
Loaded 43 clusters for 7013_m13
Loaded sniff data: 2739370 samples
Found 21709 sniff events


In [3]:
print(data.video_path)
print(data.video_offset)
print(data.total_frames)

D:\data\clickbait-motivate\bonsai\7013\m13\10232025_7013_m13.avi
1
81577


In [4]:
data.events.head()

,trial_number,timestamp,poke_left,poke_right,bonsai_centroid_x,bonsai_centroid_y,target_cell,iti,water_left,water_right,...,direction,drinking,timestamp_ms,Unnamed: 0,nose_x,nose_y,centroid_x,centroid_y,tailbase_x,tailbase_y
0,0,2025-10-23 10:45:35.305369600,False,False,849,1017,12.0,False,True,False,...,0.000000,False,15750,1,852.184387,1007.681213,755.518433,888.205139,716.478394,724.015747
1,1,2025-10-23 10:45:35.358988800,False,True,845,1013,12.0,False,True,False,...,0.785398,False,15784,2,844.134583,996.502197,748.136047,876.425720,724.424561,719.856201
2,1,2025-10-23 10:45:35.384473600,False,True,841,995,12.0,False,False,False,...,0.785291,False,15817,3,846.319231,984.431414,747.946777,864.144287,732.010010,707.830994
3,1,2025-10-23 10:45:35.411020800,False,True,845,975,12.0,False,False,False,...,1.570735,False,15850,4,848.503880,972.360631,751.457764,855.583130,728.223999,703.530334
4,1,2025-10-23 10:45:35.434713600,False,True,825,889,12.0,False,False,False,...,0.784894,False,15884,5,850.688529,960.289847,752.072571,856.032288,715.810547,700.398499


### Visualize SLEAP Tracking

In [6]:
# Visualization options
scale = 0.5  # Scaling factor for preview animation
save = False # Toggle preview
video = True  # Toggle video background
bonsai_centroid = True  # Centroid from realtime tracking

# SLEAP Nodes
nose = data.events[['nose_x', 'nose_y']].to_numpy().astype(np.int16)
cent = data.events[['centroid_x', 'centroid_y']].to_numpy().astype(np.int16)
# base = data.events[['tailbase_x', 'tailbase_y']].to_numpy().astype(np.int16)
bon_cent = data.events[['bonsai_centroid_x', 'bonsai_centroid_y']].to_numpy().astype(np.int16)


for ii in range(0, data.total_frames):
    if video:
        data.video.set(cv2.CAP_PROP_POS_FRAMES, ii + data.video_offset)  # Get current video frame
        ret, canvas = data.video.read()
    else:
        canvas = np.zeros((data.height,data.width,3), dtype=np.uint8)  # Clear canvas
    
    # Draw Bonsai centroid
    if bonsai_centroid:
        cv2.circle(img=canvas, center=bon_cent[ii], radius=12, color=(255,128,128), thickness=-1, lineType=cv2.LINE_AA)

    # Draw SLEAP nodes
    cv2.circle(img=canvas, center=nose[ii], radius=5, color=(255,0,0), thickness=-1, lineType=cv2.LINE_AA)  # Nose
    cv2.circle(img=canvas, center=cent[ii], radius=5, color=(0,255,0), thickness=-1, lineType=cv2.LINE_AA)  # Centroid
    # cv2.circle(img=canvas, center=base[ii], radius=5, color=(0,0,255), thickness=-1, lineType=cv2.LINE_AA)  # Tail base

    # Draw direction
    direction = data.events['direction']
    cv2.line(img=canvas, pt1=cent[ii], pt2=nose[ii], color=(255,255,0), thickness=2, lineType=cv2.LINE_AA)

    canvas = cv2.resize(canvas, dsize=(int(data.width*scale), int(data.height*scale)))
    canvas = cv2.rotate(canvas, cv2.ROTATE_90_CLOCKWISE) 

    if save:
        cv2.imwrite(f"S:/track-test-video/{mouse_id}_{session_id}_tracking_{ii}.png", canvas)
    else:
        cv2.imshow('SLEAP Test', canvas)
        # Wait for key press with timeout
        key = cv2.waitKey(16) & 0xFF
        # Break on 'q' key or ESC key
        if key == ord('q') or key == 27:
            break

cv2.waitKey(0)
cv2.destroyAllWindows()

In [40]:
# # Visualization options
# scale = 0.5  # Scaling factor for preview animation
# save = False # Toggle preview
# video = True  # Toggle video background
# bonsai_centroid = True  # Centroid from realtime tracking

# # SLEAP Nodes
# nose = data.events[['nose_x', 'nose_y']].to_numpy().astype(np.int16)
# cent = data.events[['centroid_x', 'centroid_y']].to_numpy().astype(np.int16)
# # base = data.events[['tailbase_x', 'tailbase_y']].to_numpy().astype(np.int16)
# bon_cent = data.events[['bonsai_centroid_x', 'bonsai_centroid_y']].to_numpy().astype(np.int16)


# for ii in range(0, data.total_frames):
#     if video:
#         data.video.set(cv2.CAP_PROP_POS_FRAMES, ii + data.video_offset)  # Get current video frame
#         ret, canvas = data.video.read()
#     else:
#         canvas = np.zeros((data.height,data.width,3), dtype=np.uint8)  # Clear canvas
    
#     # Draw Bonsai centroid
#     if bonsai_centroid:
#         cv2.circle(img=canvas, center=bon_cent[ii + data.video_offset], radius=12, color=(255,128,128), thickness=-1, lineType=cv2.LINE_AA)

#     # Draw SLEAP nodes
#     cv2.circle(img=canvas, center=nose[ii + data.video_offset], radius=5, color=(255,0,0), thickness=-1, lineType=cv2.LINE_AA)  # Nose
#     cv2.circle(img=canvas, center=cent[ii + data.video_offset], radius=5, color=(0,255,0), thickness=-1, lineType=cv2.LINE_AA)  # Centroid
#     # cv2.circle(img=canvas, center=base[ii], radius=5, color=(0,0,255), thickness=-1, lineType=cv2.LINE_AA)  # Tail base

#     canvas = cv2.resize(canvas, dsize=(int(data.width*scale), int(data.height*scale)))
#     canvas = cv2.rotate(canvas, cv2.ROTATE_90_CLOCKWISE) 

#     if save:
#         cv2.imwrite(f"S:/track-test-video/{mouse_id}_{session_id}_tracking_{ii}.png", canvas)
#     else:
#         cv2.imshow('SLEAP Test', canvas)
#         # Wait for key press with timeout
#         key = cv2.waitKey(16) & 0xFF
#         # Break on 'q' key or ESC key
#         if key == ord('q') or key == 27:
#             break

# cv2.waitKey(0)
# cv2.destroyAllWindows()